Have you ever wondered about the students who don’t sit at the top of the class—but also don’t struggle at the bottom? These are the learners who quietly attend every exam, maintain consistent (though not perfect) scores, and often slip under the radar. Educators and administrators frequently focus on top performers and those most in need of help, leaving the “quiet” students overlooked. However, discovering who they are—and understanding their potential—can be just as valuable.

Table: Student
| Column Name  | Type    |
|--------------|---------|
| student_id   | int     |
| student_name | varchar |

- **student_id** is the primary key (column with unique values) for this table.
- **student_name** is the name of the student.

---

Table: Exam
| Column Name | Type |
|-------------|------|
| exam_id     | int  |
| student_id  | int  |
| score       | int  |

- **(exam_id, student_id)** is the primary key (a combination of columns with unique values) for this table.
- Each row of this table indicates that the student with **student_id** had **score** points in the exam with ID **exam_id**.

---

### Definition of a Quiet Student
A **quiet student** is one who:
1. Has taken **at least one exam**
2. **Never** got the highest or the lowest score in **any** of those exams

---

### Task
Write a solution to report the students \(`student_id`, `student_name`\) who are quiet in **all** exams they have taken.
Do **not** return students who have **never taken any exam**.
Return the result table **ordered by** `student_id`.

---

**Input**
**Student table:**
| student_id | student_name |
|------------|--------------|
| 1          | Daniel       |
| 2          | Jade         |
| 3          | Stella       |
| 4          | Jonathan     |
| 5          | Will         |

**Exam table:**
| exam_id | student_id | score |
|---------|-----------|-------|
| 10      | 1         | 70    |
| 10      | 2         | 80    |
| 10      | 3         | 90    |
| 20      | 1         | 80    |
| 30      | 1         | 70    |
| 30      | 3         | 80    |
| 30      | 4         | 90    |
| 40      | 1         | 60    |
| 40      | 2         | 70    |
| 40      | 4         | 80    |


**Output**

| student_id | student_name |
|------------|--------------|
| 2          | Jade         |


**Explanation**

- **Exam 1**: Student 1 and Student 3 hold the lowest and highest scores, respectively.
- **Exam 2**: Student 1 holds both the highest and the lowest score (he's the only one).
- **Exam 3 & 4**: Students 1 and 4 have the lowest and highest scores, respectively.

Therefore, Students 2 and 5 have never received the highest or lowest score in any exam. However, Student 5 did not participate in **any** exam, so they are **excluded**. The only student remaining who is quiet (has taken at least one exam but never had the highest or lowest score) is **Student 2 (Jade)**.


In [1]:
import pandas as pd

data = [[1, 'Daniel'],
        [2, 'Jade'],
        [3, 'Stella'],
        [4, 'Jonathan'],
        [5, 'Will']]
student = pd.DataFrame(data,
                       columns=['student_id',
                                'student_name']).astype({'student_id':'Int64',
                                                         'student_name':'object'})

data = [[10, 1, 70],
        [10, 2, 80],
        [10, 3, 90],
        [20, 1, 80],
        [30, 1, 70],
        [30, 3, 80],
        [30, 4, 90],
        [40, 1, 60],
        [40, 2, 70],
        [40, 4, 80]]
exam = pd.DataFrame(data,
                    columns=['exam_id',
                             'student_id',
                             'score']).astype({'exam_id':'Int64',
                                               'student_id':'Int64',
                                               'score':'Int64'})
display(student, exam)

,student_id,student_name
0,1,Daniel
1,2,Jade
2,3,Stella
3,4,Jonathan
4,5,Will


,exam_id,student_id,score
0,10,1,70
1,10,2,80
2,10,3,90
3,20,1,80
4,30,1,70
5,30,3,80
6,30,4,90
7,40,1,60
8,40,2,70
9,40,4,80


**Step 1: Create a new column (highest_score) with the maximum score for each exam_id group.**
- exam.groupby('exam_id')['score'] groups the exam DataFrame by the column exam_id and focuses on the score column within each group.
- .transform('max') calculates the maximum score within each group and returns a Series with the same index as exam.
- exam['highest_score'] = ... assigns this Series to a new column called highest_score in the exam DataFrame.
- After this line, for every row in exam, highest_score will hold the maximum score of the exam that the row belongs to (i.e., the max score among all rows with the same exam_id).

**Step 2: Creates a new column (lowest_score) with the minimum score for each exam_id group.**
- Similar to the previous step, but now it calculates the minimum score within each exam_id.
- Assigns the result to a new column called lowest_score.
- After this line, for every row in exam, lowest_score will hold the minimum score of the exam that the row belongs to.

In [3]:
exam['highest_score']=exam.groupby('exam_id')['score'].transform('max')
exam['lowest_score']=exam.groupby('exam_id')['score'].transform('min')
display(exam)

,exam_id,student_id,score,highest_score,lowest_score
0,10,1,70,90,70
1,10,2,80,90,70
2,10,3,90,90,70
3,20,1,80,80,80
4,30,1,70,90,70
5,30,3,80,90,70
6,30,4,90,90,70
7,40,1,60,80,60
8,40,2,70,80,60
9,40,4,80,80,60


**Step 3: Finds all students who have either the highest or the lowest score in their exam and extracts only their student_id.**
- exam['score'] == exam['highest_score'] creates a boolean mask that is True for rows where the student’s score is equal to the highest_score for that exam_id.
- exam['score'] == exam['lowest_score'] similarly creates a boolean mask for rows where the student’s score is equal to the lowest_score for that exam_id.
- The vertical bar | (OR operator) combines these two conditions, so rows that meet either condition are selected.
- exam[...] with the combined boolean mask filters the DataFrame to keep only those rows where the student’s score is either the highest or the lowest in that exam.
- [['student_id']] keeps only the student_id column from the filtered DataFrame.
- The result is a DataFrame called max_min_students that contains the IDs of students who got the highest or lowest scores in any exam.


In [4]:
max_min_students= exam[(exam['score']== exam['highest_score']) | (exam['score']==exam['lowest_score']) ][['student_id']]
display(max_min_students)

,student_id
0,1
2,3
3,1
4,1
6,4
7,1
9,4


**Step 4: Removes any rows from exam for students identified in max_min_students.**
- max_min_students['student_id'] extracts the list of student IDs that had highest or lowest scores in any exam.
- exam['student_id'].isin(...) creates a boolean mask that is True for rows whose student_id appears in that list.
- ~ (tilde) negates the mask, so it inverts True to False and vice versa.
- exam[...] filters the DataFrame to keep only the rows where the student ID is not in max_min_students.
- As a result, exam is overwritten with a subset of itself excluding students who had the highest or lowest score in any exam.

In [5]:
exam = exam[~exam['student_id'].isin(max_min_students['student_id'])]
display(exam)

,exam_id,student_id,score,highest_score,lowest_score
1,10,2,80,90,70
8,40,2,70,80,60


**Step 5: Joins the filtered exam DataFrame with the student DataFrame using student_id as the key.**
- Merges the (filtered) exam DataFrame with the student DataFrame on the column student_id.
- This step typically brings in additional student information (e.g., student_name) from the student DataFrame into the merged DataFrame.

In [6]:
merged=exam.merge(student,on='student_id')
display(merged)


,exam_id,student_id,score,highest_score,lowest_score,student_name
0,10,2,80,90,70,Jade
1,40,2,70,80,60,Jade


**Step 6: Keeps only the student_id and student_name columns, removes duplicates, and sorts by student_id.**
- merged[['student_id', 'student_name']] selects only the specified columns from the merged DataFrame.
- .drop_duplicates() removes any duplicate rows, ensuring each (student_id, student_name) pair appears only once.
- .sort_values(by='student_id') sorts the rows by the student_id in ascending order.
- The resulting merged DataFrame now contains a clean list of students (with their names) who did not have the highest or lowest scores in any exam, sorted by student_id.

In [7]:
merged = merged[['student_id','student_name']].drop_duplicates().sort_values(by='student_id')
display(merged)

,student_id,student_name
0,2,Jade


References: [1] https://leetcode.com/problems/find-the-quiet-students-in-all-exams/description/?lang=pythondata
